Rudolph City Visit Optimization Kaggle Contest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [2]:
data = pd.read_csv("cities.csv")
print(data.shape)
print(data.head(10))

(197769, 3)
   CityId            X            Y
0       0   316.836739  2202.340707
1       1  4377.405972   336.602082
2       2  3454.158198  2820.053011
3       3  4688.099298  2935.898056
4       4  1010.696952  3236.750989
5       5  2474.230877  1435.514651
6       6  1029.277795  2721.800952
7       7  3408.887685   199.585793
8       8  1304.006125  2657.427246
9       9  4211.525725  2294.595208


New function to figure out numbers are prime

In [3]:
def is_prime(n):
    if n > 2:
        i = 2
        while i ** 2 <= n:
            if n % i:
                i += 1
            else:
                return False
    elif n != 2:
        return False
    return True

In [4]:
data['Prime'] = data.CityId.apply(is_prime)
print(data.head(5))

   CityId            X            Y  Prime
0       0   316.836739  2202.340707  False
1       1  4377.405972   336.602082  False
2       2  3454.158198  2820.053011   True
3       3  4688.099298  2935.898056   True
4       4  1010.696952  3236.750989  False


In [5]:
def distance_traveled(data,route):
    previous_city = route[0]
    traveled_distance = 0
    penalty_distance = 0
    step = 1
    for city_number in route[1:]:
        next_city = city_number
        traveled_distance += np.sqrt((data.X[city_number] - data.X[previous_city])**2 + (data.Y[city_number] - data.Y[previous_city])**2) * (1 + 0.1 * (step % 10 ==0)*int(data.Prime[previous_city]))
        # For now no penalties involved
        step += 1
        previous_city = next_city
    return traveled_distance


In [6]:
route_basic = list(data.CityId[:].append(pd.Series([0])))
route_count = route_basic[:-1]

In [7]:
distance_basic = distance_traveled(data,route_basic)

In [8]:
print(distance_basic)

444429424.3489193


Distance function seems to be working (at least well enough for testing) 
Now try to optimize route a little bit more

Try sorting things a little bit: Start with just sorting by X and Y

In [9]:
data_sortedX = data.sort_values(by=['X'])
data_sortedY = data.sort_values(by=['Y'])

In [10]:
route_X_sorted = list(data_sortedX.CityId[:].append(pd.Series([0])))
route_Y_sorted = list(data_sortedY.CityId[:].append(pd.Series([0])))

In [11]:
distance_X_sort = distance_traveled(data,route_X_sorted)
distance_Y_sort = distance_traveled(data,route_Y_sorted)
# probably have to rework this since 0 isn't first in list, but oh well

In [12]:
print("Basic Order: ", distance_basic)
print("X Sort:      ",distance_X_sort)
print("Y Sort:      ",distance_Y_sort)

Basic Order:  444429424.3489193
X Sort:       194886963.41873947
Y Sort:       319845121.49794346


All of the numbers are in the top right quadrant (all positive)
Create a third category (distance from origin) and sort by that

In [13]:
def distance_origin(X,Y):
    Z = np.sqrt(X**2 + Y**2)
    return Z

In [14]:
data["Z"] = distance_origin(data.X,data.Y)

In [15]:
data_sortedZ = data.sort_values(by=['Z'])
route_Z_sorted = list(data_sortedZ.CityId[:].append(pd.Series([0])))
distance_Z_sort = distance_traveled(data,route_Z_sorted)
print("Z sort: ", distance_Z_sort)

Z sort:  223211526.34047332


Make a Nearest Neighbors Table for each point (5 each?)

In [ ]:
nearest_5 = np.zeros((len(route_count),5))
xy = np.array([data.X.values,data.Y.values]).T
for i in route_count:
    current_xy = np.array([data.X[i],data.Y[i]])
    distances = np.sqrt(((xy - current_xy)**2).sum(-1))
    nearest_5[i,] = distances.argsort()[:5]
    if i % 1000 == 0:
        print(i)
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000


In [ ]:
nearest_5 = nearest_5.astype(int)
print(nearest_5[0:10,:]) # Just to see if it worked. Didn't put in a condition for not being itself
nearest_5_update = nearest_5[:,1:] ## removed the self in closest condition
print(nearest_5_update[0:10,:])

Greedy Optimization for figuring out what paths to take

In [ ]:
ids = data.CityId.values[1:].tolist()
NN_greedy_path = [0,]
random_usage = 0
while len(ids_city>0):
    current_step = NN_greedy_path[-1]
    if nearest_5_update[current_step,0] in ids:
        next_step = nearest_5_update[current_step,0]
    elif nearest_5_update[current_step,1] in ids:
        next_step = nearest_5_update[current_step,1]
    elif nearest_5_update[current_step,2] in ids:
        next_step = nearest_5_update[current_step,2]
    elif nearest_5_update[current_step,3] in ids:
        next_step = nearest_5_update[current_step,3]
    else:
        next_step = random.choice(ids)
        random_usage += 1
        print(random_usage)
    NN_greedy_path.append(next_step)
    ids.remove(next_step)
NN_greedy_path.append(0)